# Feature Engineeing
1. Analyse des Données

2. Génération de Caractéristiques

3. Sélection de Caractéristiques 

4. Encodage des Caractéristiques 

5. Normalisation ou Standardisation 
## Importation des Bibliothèques Nécessaires

In [1]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os
import pickle

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# Fonctions
from fonctions import *

# Configuration de l'affichage pour les graphiques
%matplotlib inline
sns.set(style="whitegrid")

In [2]:
path_data = '../data/raw/'
path_data_processed = "../data/processed/"

In [3]:
os.listdir(path_data)

['application_test.csv',
 'application_train.csv',
 'bureau.csv',
 'bureau_balance.csv',
 'credit_card_balance.csv',
 'HomeCredit_columns_description.csv',
 'installments_payments.csv',
 'POS_CASH_balance.csv',
 'previous_application.csv',
 'sample_submission.csv']

In [4]:
os.listdir(path_data_processed)

['processed_data_test.gzip', 'processed_data_train.csv']

In [5]:
# Tables principales
data_train = pd.read_csv(os.path.join(path_data, 'application_train.csv'))
data_test = pd.read_csv(os.path.join(path_data, 'application_test.csv'))

In [6]:
data_bureau = pd.read_csv(os.path.join(path_data, 'bureau.csv'))
data_bureau_balance = pd.read_csv(os.path.join(path_data, 'bureau_balance.csv'))
data_cc_bal = pd.read_csv(os.path.join(path_data, 'credit_card_balance.csv'))
data_ins_payments = pd.read_csv(os.path.join(path_data, 'installments_payments.csv'))
data_pos_cash_bal = pd.read_csv(os.path.join(path_data, 'POS_CASH_balance.csv'))
data_prev_app = pd.read_csv(os.path.join(path_data, 'previous_application.csv'))

Dans le cadre du projet sur le risque de défaut de crédit chez Home Credit, le prétraitement et l'ingénierie des caractéristiques sont essentiels pour développer un modèle prédictif fiable. Ce processus est largement inspiré par un Kernel de Kaggle, disponible via ce [lien](https://www.kaggle.com/code/jsaguiar/lightgbm-with-simple-features/script), qui propose une approche méthodique pour créer des caractéristiques à partir des données. Voici les principaux points et techniques utilisés dans ce kernel :

**Techniques de Feature Engineering Employées**
* **Ratios de Caractéristiques Importantes :** Nous allons manipuler les données en divisant ou soustrayant des caractéristiques importantes pour obtenir des taux qui pourraient révéler des informations plus nuancées sur la capacité de remboursement des clients. Par exemple, le ratio entre l'annuité et le revenu d'un client peut nous donner des insights sur sa stabilité financière et sa capacité à maintenir des paiements réguliers.

* **Encodage One-hot pour les Caractéristiques Catégorielles :** Les caractéristiques catégorielles seront transformées en utilisant l'encodage One-hot. Cette méthode crée de nouvelles colonnes indiquant la présence de chaque valeur possible dans les caractéristiques originales, transformant ainsi les données catégorielles en un format numérique qui peut être traité efficacement par les algorithmes de machine learning.

*Mise à jour du 16/06/2018 :*

*Ajout de la caractéristique "Taux de Paiement" : Une nouvelle caractéristique reflétant le ratio entre l'annuité du prêt et le revenu annuel du client a été intégrée, permettant ainsi une meilleure évaluation de la capacité de remboursement du client.*

*Suppression de l'index des caractéristiques : Les indices ont été retirés des caractéristiques utilisées pour le modèle afin de simplifier le traitement des données et de prévenir les erreurs potentielles liées à des indices inappropriés comme variables explicatives.*



In [7]:
# Variables utiles
TARGET = 'TARGET' 
ID = 'SK_ID_CURR'

## Jointre de la table `application_test` vers `application_train`

In [8]:
data_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
data_train.shape

(307511, 122)

In [10]:
data_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
data_test.shape

(48744, 121)

In [12]:
# Liste des IDs
list_ids = data_test[ID].to_list()
len(list_ids)

48744

In [13]:
# Jointure des tables
data_train = pd.concat([data_train, data_test])
data_train.shape[0]

356255

## Nouvelle Table `application_train`

### [1. Nettoyage des Données](#clean)
* **Suppression des entrées où le sexe est indiqué par 'XNA' :** Ces données peuvent représenter des erreurs de saisie ou des cas atypiques qui pourraient fausser l'analyse et le modèle de prédiction. En les supprimant, nous améliorons la qualité et la fiabilité de notre modèle.

### [2. Traitement des Anomalies](#traitement)
* **Remplacement des anomalies dans 'DAYS_EMPLOYED' par NaN :** Les valeurs anormales dans cette caractéristique (par exemple, un nombre de jours d'emploi trop élevé qui n'est pas plausible) sont remplacées par des valeurs NaN pour éviter de fausser les analyses et les prédictions. Cela permet de traiter ces anomalies de façon cohérente lors de l'imputation des données manquantes.

### [3. Création de Nouvelles Caractéristiques](#create_feature)
* **Création de 5 nouvelles caractéristiques :** Ces nouvelles caractéristiques peuvent inclure des ratios ou des combinaisons de caractéristiques existantes qui sont jugées importantes pour prédire le risque de défaut de crédit. La création de nouvelles caractéristiques est basée sur notre compréhension des facteurs qui influencent le risque de crédit et peut aider à améliorer significativement la performance du modèle.

### [4. Encodage des Caractéristiques](#encode)
* **Encodage binaire des types de contrats, du sexe, de la possession de voiture, de la propriété immobilière et de l'état d'urgence :** Transformer ces variables catégorielles en formats binaires (0 ou 1) simplifie le modèle en réduisant le nombre de dimensions et facilite l'interprétation des résultats.

* **Encodage One-hot des autres variables catégorielles :** L'encodage One-hot est utilisé pour les variables catégorielles qui contiennent plus de deux catégories. Cela permet de transformer chaque catégorie en une nouvelle caractéristique binaire, ce qui aide les algorithmes de machine learning à mieux traiter et interpréter ces informations sans introduire d'ordre artificiel.

<a class="anchor" id="clean"></a>

In [14]:
# Identifier les entrées avec un genre non spécifié ('XNA')
index_xna = data_train[data_train['CODE_GENDER'] == 'XNA'].index

# Supprimer ces entrées du DataFrame
data_train = data_train.drop(index_xna)
print(f"CODE_GENDER: {len(index_xna)} entrées indéfinies (XNA) supprimées")

CODE_GENDER: 4 entrées indéfinies (XNA) supprimées


In [15]:
data_train.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,356249.000000,307507.00000,356249.000000,3.562490e+05,3.562490e+05,356213.000000,3.559710e+05,356249.000000,356249.000000,356249.00000,...,356249.000000,356249.000000,356249.000000,356249.000000,308682.000000,308682.000000,308682.000000,308682.000000,308682.000000,308682.000000
mean,278128.342126,0.08073,0.414314,1.701155e+05,5.877671e+05,27425.552488,5.280194e+05,0.020917,-16041.279984,64318.37965,...,0.007231,0.000514,0.000438,0.000289,0.005809,0.006282,0.029995,0.231692,0.304397,1.911534
std,102842.112712,0.27242,0.720377,2.235086e+05,3.986250e+05,14732.837932,3.660659e+05,0.013915,4358.810128,141706.44918,...,0.084727,0.022659,0.020921,0.017001,0.079737,0.104251,0.191375,0.855941,0.786919,1.865325
min,100001.000000,0.00000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000253,-25229.000000,-17912.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189065.000000,0.00000,0.000000,1.125000e+05,2.700000e+05,16731.000000,2.340000e+05,0.010006,-19676.000000,-2781.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278128.000000,0.00000,0.000000,1.530000e+05,5.002110e+05,25078.500000,4.500000e+05,0.018850,-15755.000000,-1224.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367192.000000,0.00000,1.000000,2.025000e+05,7.975575e+05,34960.500000,6.750000e+05,0.028663,-12425.000000,-290.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,456255.000000,1.00000,20.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7338.000000,365243.00000,...,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


<a class="anchor" id="traitement"></a>

In [16]:
# Trouver la valeur anomalie pour les jours employés
anom = data_train['DAYS_EMPLOYED'].max()

# Remplacer cette valeur anormale par NaN
data_train = data_train.replace({anom: np.nan})
print(f"DAYS_EMPLOYED: remplacé {anom} par NaN.")

DAYS_EMPLOYED: remplacé 365243 par NaN.


In [17]:
data_train.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,356248.000000,307507.00000,356249.000000,3.562490e+05,3.562490e+05,356213.000000,3.559710e+05,356249.000000,356249.000000,291601.000000,...,356249.000000,356249.000000,356249.000000,356249.000000,308682.000000,308682.000000,308682.000000,308682.000000,308682.000000,308682.000000
mean,278128.097592,0.08073,0.414314,1.701155e+05,5.877671e+05,27425.552488,5.280194e+05,0.020917,-16041.279984,-2396.668845,...,0.007231,0.000514,0.000438,0.000289,0.005809,0.006282,0.029995,0.231692,0.304397,1.911534
std,102842.153484,0.27242,0.720377,2.235086e+05,3.986250e+05,14732.837932,3.660659e+05,0.013915,4358.810128,2334.458385,...,0.084727,0.022659,0.020921,0.017001,0.079737,0.104251,0.191375,0.855941,0.786919,1.865325
min,100001.000000,0.00000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000253,-25229.000000,-17912.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189064.750000,0.00000,0.000000,1.125000e+05,2.700000e+05,16731.000000,2.340000e+05,0.010006,-19676.000000,-3200.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278127.500000,0.00000,0.000000,1.530000e+05,5.002110e+05,25078.500000,4.500000e+05,0.018850,-15755.000000,-1663.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367192.250000,0.00000,1.000000,2.025000e+05,7.975575e+05,34960.500000,6.750000e+05,0.028663,-12425.000000,-780.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,456255.000000,1.00000,20.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7338.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


<a class="anchor" id="create_feature"></a>

In [18]:
# Créer des nouvelles caractéristiques

# Pourcentage de jours employés par rapport à l'âge en jours
data_train['DAYS_EMPLOYED_PERC'] = data_train['DAYS_EMPLOYED'] / data_train['DAYS_BIRTH']

# Ratio du revenu annuel total sur le montant du crédit
data_train['INCOME_CREDIT_PERC'] = data_train['AMT_INCOME_TOTAL'] / data_train['AMT_CREDIT']

# Revenu annuel total divisé par le nombre de membres de la famille
data_train['INCOME_PER_PERSON'] = data_train['AMT_INCOME_TOTAL'] / data_train['CNT_FAM_MEMBERS']

# Pourcentage de l'annuité par rapport au revenu annuel total
data_train['ANNUITY_INCOME_PERC'] = data_train['AMT_ANNUITY'] / data_train['AMT_INCOME_TOTAL']

# Taux de paiement calculé comme le ratio de l'annuité sur le montant total du crédit
data_train['PAYMENT_RATE'] = data_train['AMT_ANNUITY'] / data_train['AMT_CREDIT']


In [19]:
data_train.sample(3)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE
171175,298382.0,0.0,Cash loans,M,N,Y,1,216000.0,582804.0,31747.5,...,0.0,0.0,0.0,0.0,0.0,0.148549,0.370622,72000.0,0.146979,0.054474
27334,131768.0,0.0,Cash loans,F,N,Y,0,67500.0,284400.0,10345.5,...,0.0,0.0,0.0,0.0,4.0,NaN,0.237342,33750.0,0.153267,0.036377
208560,341734.0,0.0,Cash loans,F,N,N,0,112500.0,345843.0,32499.0,...,NaN,NaN,NaN,NaN,NaN,0.058059,0.325292,56250.0,0.288880,0.093970


<a class="anchor" id="encode"></a>

In [20]:
def encode_binary_features(dataframe):
    """
    Identifie et encode les caractéristiques binaires dans le DataFrame en utilisant la factorisation.

    Args:
    dataframe (pd.DataFrame): Le DataFrame sur lequel opérer.

    Returns:
    pd.DataFrame: DataFrame avec les caractéristiques binaires encodées.
    """
    # Identification des caractéristiques binaires (type object avec deux valeurs uniques)
    binary_features = [col for col in dataframe.select_dtypes(include='object').columns if dataframe[col].nunique() == 2]
    print(f"Caractéristiques binaires identifiées : {binary_features}")

    # Encodage des caractéristiques binaires en utilisant la factorisation
    for col in binary_features:
        dataframe[col], uniques = pd.factorize(dataframe[col])
        print(f"Encodage réalisé pour {col} - valeurs uniques : {uniques}")

    # Affichage des premières lignes des caractéristiques binaires après encodage
    return dataframe[binary_features]

# Utilisation de la fonction pour encoder les caractéristiques binaires
encoded_bin_features = encode_binary_features(data_train)
encoded_bin_features.head()

Caractéristiques binaires identifiées : ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'EMERGENCYSTATE_MODE']
Encodage réalisé pour NAME_CONTRACT_TYPE - valeurs uniques : Index(['Cash loans', 'Revolving loans'], dtype='object')
Encodage réalisé pour CODE_GENDER - valeurs uniques : Index(['M', 'F'], dtype='object')
Encodage réalisé pour FLAG_OWN_CAR - valeurs uniques : Index(['N', 'Y'], dtype='object')
Encodage réalisé pour FLAG_OWN_REALTY - valeurs uniques : Index(['Y', 'N'], dtype='object')
Encodage réalisé pour EMERGENCYSTATE_MODE - valeurs uniques : Index(['No', 'Yes'], dtype='object')


,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,EMERGENCYSTATE_MODE
0,0,0,0,0,0
1,0,1,0,1,0
2,1,0,1,0,-1
3,0,1,0,0,-1
4,0,0,0,0,-1


In [21]:
# Application de l'encodage one-hot sur data_train
data_train, cat_train_cols = one_hot_encoder(data_train, nan_as_category=False)

print(f"Nouvelles colonnes ajoutées par l'encodage one-hot: {cat_train_cols}")
data_train.head()

Nouvelles colonnes ajoutées par l'encodage one-hot: ['NAME_TYPE_SUITE_Children', 'NAME_TYPE_SUITE_Family', 'NAME_TYPE_SUITE_Group of people', 'NAME_TYPE_SUITE_Other_A', 'NAME_TYPE_SUITE_Other_B', 'NAME_TYPE_SUITE_Spouse, partner', 'NAME_TYPE_SUITE_Unaccompanied', 'NAME_INCOME_TYPE_Businessman', 'NAME_INCOME_TYPE_Commercial associate', 'NAME_INCOME_TYPE_Maternity leave', 'NAME_INCOME_TYPE_Pensioner', 'NAME_INCOME_TYPE_State servant', 'NAME_INCOME_TYPE_Student', 'NAME_INCOME_TYPE_Unemployed', 'NAME_INCOME_TYPE_Working', 'NAME_EDUCATION_TYPE_Academic degree', 'NAME_EDUCATION_TYPE_Higher education', 'NAME_EDUCATION_TYPE_Incomplete higher', 'NAME_EDUCATION_TYPE_Lower secondary', 'NAME_EDUCATION_TYPE_Secondary / secondary special', 'NAME_FAMILY_STATUS_Civil marriage', 'NAME_FAMILY_STATUS_Married', 'NAME_FAMILY_STATUS_Separated', 'NAME_FAMILY_STATUS_Single / not married', 'NAME_FAMILY_STATUS_Unknown', 'NAME_FAMILY_STATUS_Widow', 'NAME_HOUSING_TYPE_Co-op apartment', 'NAME_HOUSING_TYPE_House / 

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden
0,100002.0,1.0,0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,0,0,0,0,0,1,0
1,100003.0,0.0,0,1,0,1,0,270000.0,1293502.5,35698.5,...,1,0,0,1,0,0,0,0,0,0
2,100004.0,0.0,1,0,1,0,0,67500.0,135000.0,6750.0,...,0,0,0,0,0,0,0,0,0,0
3,100006.0,0.0,0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,0,0,0,0,0,0
4,100007.0,0.0,0,0,0,0,0,121500.0,513000.0,21865.5,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# Copy de data_train pour nos futures merges
data_train_clean = data_train.copy()

## Table `bureau` et `bureau_balancee`

1. **Encodage One-hot des variables catégorielles**

2. **Jointure de bureau_balance avec bureau par SK_ID_BUREAU :** `bureau_balance` contient des informations mensuelles sur les crédits antérieurs des clients conservés dans le bureau de crédit, tandis que `bureau` contient des résumés de crédits antérieurs. Cette fusion est réalisée en utilisant l'identifiant de crédit (`SK_ID_BUREAU`), permettant ainsi d'associer chaque entrée mensuelle de `bureau_balance` avec son crédit respectif dans `bureau`. Cette étape permet d'agréger des informations détaillées pour une vue plus complète de l'historique de crédit des clients.

3. **Créer une caractéristique : `BURO_COUNT` (nombre de prêts précédents) :** Cette caractéristique est calculée en comptant le nombre d'entrées pour chaque client dans la table bureau, ce qui donne une indication du nombre de fois qu'un client a été impliqué dans des activités de prêt avant la demande actuelle. Cela peut être un indicateur de la fiabilité ou de l'activité de crédit du client.

4. **Jointure de bureau avec data_train_clean par ID :** Après avoir préparé la table `bureau` en ajoutant de nouvelles informations et caractéristiques, la table est ensuite fusionnée avec `data_train_clean` (la table principale contenant les demandes de prêt). Cette fusion est basée sur l'identifiant unique du client (`SK_ID_CURR`). L'intégration de ces données permet de renforcer le modèle prédictif avec des informations historiques détaillées sur le crédit, potentiellement utiles pour prédire le risque de défaut de crédit.

In [23]:
data_bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [24]:
data_bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


### 1. Encodage One-hot des variables catégorielles

In [25]:
# Application de l'encodage one-hot sur data_bureau
data_bureau, cat_bureau_cols = one_hot_encoder(data_bureau, nan_as_category=False)

print(f"Nouvelles colonnes ajoutées par l'encodage one-hot: {cat_bureau_cols}")
data_bureau.head()

Nouvelles colonnes ajoutées par l'encodage one-hot: ['CREDIT_ACTIVE_Active', 'CREDIT_ACTIVE_Bad debt', 'CREDIT_ACTIVE_Closed', 'CREDIT_ACTIVE_Sold', 'CREDIT_CURRENCY_currency 1', 'CREDIT_CURRENCY_currency 2', 'CREDIT_CURRENCY_currency 3', 'CREDIT_CURRENCY_currency 4', 'CREDIT_TYPE_Another type of loan', 'CREDIT_TYPE_Car loan', 'CREDIT_TYPE_Cash loan (non-earmarked)', 'CREDIT_TYPE_Consumer credit', 'CREDIT_TYPE_Credit card', 'CREDIT_TYPE_Interbank credit', 'CREDIT_TYPE_Loan for business development', 'CREDIT_TYPE_Loan for purchase of shares (margin lending)', 'CREDIT_TYPE_Loan for the purchase of equipment', 'CREDIT_TYPE_Loan for working capital replenishment', 'CREDIT_TYPE_Microloan', 'CREDIT_TYPE_Mobile operator loan', 'CREDIT_TYPE_Mortgage', 'CREDIT_TYPE_Real estate loan', 'CREDIT_TYPE_Unknown type of loan']


,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,...,CREDIT_TYPE_Interbank credit,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan
0,215354,5714462,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,215354,5714463,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,...,0,0,0,0,0,0,0,0,0,0
2,215354,5714464,-203,0,528.0,NaN,NaN,0,464323.5,NaN,...,0,0,0,0,0,0,0,0,0,0
3,215354,5714465,-203,0,NaN,NaN,NaN,0,90000.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,215354,5714466,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# Application de l'encodage one-hot sur data_bureau_balance
data_bureau_balance, cat_bureau_bal_cols = one_hot_encoder(data_bureau_balance, nan_as_category=False)

print(f"Nouvelles colonnes ajoutées par l'encodage one-hot: {cat_bureau_bal_cols}")
data_bureau_balance.head()

Nouvelles colonnes ajoutées par l'encodage one-hot: ['STATUS_0', 'STATUS_1', 'STATUS_2', 'STATUS_3', 'STATUS_4', 'STATUS_5', 'STATUS_C', 'STATUS_X']


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X
0,5715448,0,0,0,0,0,0,0,1,0
1,5715448,-1,0,0,0,0,0,0,1,0
2,5715448,-2,0,0,0,0,0,0,1,0
3,5715448,-3,0,0,0,0,0,0,1,0
4,5715448,-4,0,0,0,0,0,0,1,0


Pour le bloc du code suivant, nous définissons et appliquons des opérations d'agrégation sur les données de la table `bureau_balance`.

1. **Définition des agrégations**: Un dictionnaire nommé `agg_bureau_balance`s est créé. Ce dictionnaire spécifie les statistiques à calculer pour chaque colonne pertinente dans la table bureau_balance. Ici, la colonne `MONTHS_BALANCE` subira les opérations suivantes :

* `min` : Calcule la valeur minimum de `MONTHS_BALANCE`, qui peut indiquer la plus ancienne interaction du client avec le crédit dans les données.
* `max` : Calcule la valeur maximum de `MONTHS_BALANCE`, reflétant la transaction la plus récente.
* `size` : Compte le nombre de mois durant lesquels des informations sont disponibles pour chaque crédit, ce qui offre une perspective de la durée pendant laquelle les informations ont été suivies.

2. **Ajout d'agrégations pour les colonnes encodées :** Pour chaque colonne précédemment encodée et stockée dans la liste `agg_bureau_balance`, une agrégation de type `mean` (moyenne) est appliquée. Cela signifie que pour chaque colonne catégorielle traitée par l'encodage, nous calculons la moyenne pour regrouper les informations par `SK_ID_BUREAU`. Ceci est particulièrement utile pour résumer des données catégorielles en un seul chiffre représentatif par catégorie dans les données agrégées.

In [27]:
agg_bureau_balance = {'MONTHS_BALANCE': ['min', 'max', 'size']}

for cat in cat_bureau_bal_cols:
    agg_bureau_balance[cat] = ['mean']
print(agg_bureau_balance)

{'MONTHS_BALANCE': ['min', 'max', 'size'], 'STATUS_0': ['mean'], 'STATUS_1': ['mean'], 'STATUS_2': ['mean'], 'STATUS_3': ['mean'], 'STATUS_4': ['mean'], 'STATUS_5': ['mean'], 'STATUS_C': ['mean'], 'STATUS_X': ['mean']}


Nous traitons maintenant les données groupées de la table `bureau_balance` en utilisant les spécifications définies précédemment.

In [28]:
# Grouper les données de bureau_balance par SK_ID_BUREAU et appliquer les opérations d'agrégation définies
bureau_balance_clean = data_bureau_balance.groupby('SK_ID_BUREAU').agg(agg_bureau_balance)

# Renommer les colonnes du DataFrame groupé
bureau_balance_clean.columns = flatten_and_rename_columns(bureau_balance_clean)
bureau_balance_clean.head()

,MONTHS_BALANCE_MIN,MONTHS_BALANCE_MAX,MONTHS_BALANCE_SIZE,STATUS_0_MEAN,STATUS_1_MEAN,STATUS_2_MEAN,STATUS_3_MEAN,STATUS_4_MEAN,STATUS_5_MEAN,STATUS_C_MEAN,STATUS_X_MEAN
SK_ID_BUREAU,,,,,,,,,,,
5001709,-96,0,97,0.000000,0.0,0.0,0.0,0.0,0.0,0.886598,0.113402
5001710,-82,0,83,0.060241,0.0,0.0,0.0,0.0,0.0,0.578313,0.361446
5001711,-3,0,4,0.750000,0.0,0.0,0.0,0.0,0.0,0.000000,0.250000
5001712,-18,0,19,0.526316,0.0,0.0,0.0,0.0,0.0,0.473684,0.000000
5001713,-21,0,22,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000


### 2. Jointure de la table `data_bureau` et `data_bureau_balance` groupé 

1. **Fusion des données :** Les données agrégées de `bureau_balance` sont fusionnées avec les données de `bureau` en utilisant `SK_ID_BUREAU` comme clé. La fusion est effectuée en utilisant une jointure à gauche `(how='left')`, ce qui signifie que toutes les lignes de `data_bureau` sont conservées et les colonnes correspondantes de `bureau_balance_groupby` sont ajoutées.

2. **Suppression de la colonne `SK_ID_BUREAU` :** Après la fusion, la colonne `SK_ID_BUREAU` dans `data_bureau_balance` n'est plus nécessaire car elle a été utilisée pour effectuer la fusion. Elle est donc supprimée pour nettoyer les données.

In [29]:
data_bureau = data_bureau.join(bureau_balance_clean, how='left', on='SK_ID_BUREAU')

data_bureau_balance = data_bureau_balance.drop(['SK_ID_BUREAU'], axis=1)
data_bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,...,MONTHS_BALANCE_MAX,MONTHS_BALANCE_SIZE,STATUS_0_MEAN,STATUS_1_MEAN,STATUS_2_MEAN,STATUS_3_MEAN,STATUS_4_MEAN,STATUS_5_MEAN,STATUS_C_MEAN,STATUS_X_MEAN
0,215354,5714462,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215354,5714463,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,5714464,-203,0,528.0,NaN,NaN,0,464323.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,5714465,-203,0,NaN,NaN,NaN,0,90000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,5714466,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3. Créer une caractéristique : `BURO_COUNT` (nombre de prêts précédents) :

**1. Agrégation des Données du Bureau (bureau) :**

* **Pourquoi ?:** La table bureau **contient des informations sur les crédits passés de chaque client**. Chaque client peut avoir **plusieurs entrées** dans cette table, représentant **différents crédits**. Pour consolider cette information, nous utilisons l'agrégation pour **résumer les caractéristiques des crédits passés en une seule ligne par client**.

**2. Renommage et Aplatissement des Colonnes :**

* **Pourquoi ? :** Après l'agrégation, les noms de colonnes résultants peuvent être des tuples représentant les différentes statistiques (e.g., (`'AMT_CREDIT_SUM', 'mean'`)). Ces noms de colonnes doivent être aplanis et renommés pour être plus compréhensibles et utilisables.

**3. Comptage des Prêts Précédents :**

* **Pourquoi ? :** Le nombre total de crédits qu'un client a eus par le passé peut être un **indicateur important de son comportement de crédit**. *Par exemple, un client ayant de nombreux crédits passés pourrait être plus expérimenté dans la gestion de ses dettes, ou au contraire, pourrait avoir des difficultés s'il a de nombreux crédits en cours.*

In [30]:
# Bureau numeric features
agg_bureau = {'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
                   'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
                   'DAYS_CREDIT_UPDATE': ['mean'],
                   'CREDIT_DAY_OVERDUE': ['max', 'mean'],
                   'AMT_CREDIT_MAX_OVERDUE': ['mean'],
                   'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
                   'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
                   'AMT_CREDIT_SUM_OVERDUE': ['mean'],
                   'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
                   'AMT_ANNUITY': ['max', 'mean'],
                   'CNT_CREDIT_PROLONG': ['sum'],
                   'MONTHS_BALANCE_MIN': ['min'],
                   'MONTHS_BALANCE_MAX': ['max'],
                   'MONTHS_BALANCE_SIZE': ['mean', 'sum']}

# # Bureau categorical features
for cat in cat_bureau_cols:
    agg_bureau[cat] = ['mean']
for cat in cat_bureau_bal_cols:
    agg_bureau[cat + "_MEAN"] = ['mean']

In [31]:
# Traitement des données groupées de la table bureau en utilisant les spécifications définies précédemment.
data_bureau_clean = data_bureau.groupby(ID).agg(agg_bureau)

data_bureau_clean.columns = flatten_and_rename_columns(data_bureau_clean, prefix='BURO_')
data_bureau_clean.head()

,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_MEAN,BURO_DAYS_CREDIT_VAR,BURO_DAYS_CREDIT_ENDDATE_MIN,BURO_DAYS_CREDIT_ENDDATE_MAX,BURO_DAYS_CREDIT_ENDDATE_MEAN,BURO_DAYS_CREDIT_UPDATE_MEAN,BURO_CREDIT_DAY_OVERDUE_MAX,BURO_CREDIT_DAY_OVERDUE_MEAN,...,BURO_CREDIT_TYPE_Real estate loan_MEAN,BURO_CREDIT_TYPE_Unknown type of loan_MEAN,BURO_STATUS_0_MEAN_MEAN,BURO_STATUS_1_MEAN_MEAN,BURO_STATUS_2_MEAN_MEAN,BURO_STATUS_3_MEAN_MEAN,BURO_STATUS_4_MEAN_MEAN,BURO_STATUS_5_MEAN_MEAN,BURO_STATUS_C_MEAN_MEAN,BURO_STATUS_X_MEAN_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-1572,-49,-735.000000,240043.666667,-1329.0,1778.0,82.428571,-93.142857,0,0.0,...,0.0,0.0,0.336651,0.007519,0.0,0.0,0.0,0.0,0.441240,0.214590
100002,-1437,-103,-874.000000,186150.000000,-1072.0,780.0,-349.000000,-499.875000,0,0.0,...,0.0,0.0,0.406960,0.255682,0.0,0.0,0.0,0.0,0.175426,0.161932
100003,-2586,-606,-1400.750000,827783.583333,-2434.0,1216.0,-544.500000,-816.000000,0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100004,-1326,-408,-867.000000,421362.000000,-595.0,-382.0,-488.500000,-532.000000,0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100005,-373,-62,-190.666667,26340.333333,-128.0,1324.0,439.333333,-54.333333,0,0.0,...,0.0,0.0,0.735043,0.000000,0.0,0.0,0.0,0.0,0.128205,0.136752


In [32]:
data_bureau_clean['BURO_COUNT'] = data_bureau.groupby(ID).size()

### 4. Jointure de bureau avec data_train_clean par ID :

In [33]:
data_train_clean = data_train_clean.join(data_bureau_clean, how='left', on=ID)
data_train_clean.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,BURO_CREDIT_TYPE_Unknown type of loan_MEAN,BURO_STATUS_0_MEAN_MEAN,BURO_STATUS_1_MEAN_MEAN,BURO_STATUS_2_MEAN_MEAN,BURO_STATUS_3_MEAN_MEAN,BURO_STATUS_4_MEAN_MEAN,BURO_STATUS_5_MEAN_MEAN,BURO_STATUS_C_MEAN_MEAN,BURO_STATUS_X_MEAN_MEAN,BURO_COUNT
0,100002.0,1.0,0,0,0,0,0,202500.0,406597.5,24700.5,...,0.0,0.40696,0.255682,0.0,0.0,0.0,0.0,0.175426,0.161932,8.0
1,100003.0,0.0,0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2,100004.0,0.0,1,0,1,0,0,67500.0,135000.0,6750.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,100006.0,0.0,0,1,0,0,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007.0,0.0,0,0,0,0,0,121500.0,513000.0,21865.5,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


## Table `credit_card_balance`

1. **Encodage One-hot des variables catégorielles**

2. **Ajout d'une caractéristique :** `CC_COUNT` (nombre de comptes de cartes de crédit) : Cette nouvelle caractéristique est créée en **comptant le nombre de comptes de cartes de crédit pour chaque client dans la table `credit_card_balance`**.

3. **Jointure de `credit_card_balance` avec `data_train_clean` par `ID` :** Une fois que la table `credit_card_balance` est préparée et enrichie avec les nouvelles caractéristiques, elle est fusionnée avec la table principale `application_train` en utilisant l'identifiant unique du client (`SK_ID_CURR`). Cette fusion permet d'intégrer les informations détaillées sur les comptes de cartes de crédit des clients, renforçant ainsi le modèle prédictif avec des données supplémentaires qui peuvent aider à évaluer le risque de défaut de crédit.

In [34]:
data_cc_bal.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [35]:
# Application de l'encodage one-hot sur data_cc_bal
data_cc_bal, cat_cc_bal_cols = one_hot_encoder(data_cc_bal, nan_as_category=False)

print(f"Nouvelles colonnes ajoutées par l'encodage one-hot: {cat_cc_bal_cols}")
data_cc_bal.head()

Nouvelles colonnes ajoutées par l'encodage one-hot: ['NAME_CONTRACT_STATUS_Active', 'NAME_CONTRACT_STATUS_Approved', 'NAME_CONTRACT_STATUS_Completed', 'NAME_CONTRACT_STATUS_Demand', 'NAME_CONTRACT_STATUS_Refused', 'NAME_CONTRACT_STATUS_Sent proposal', 'NAME_CONTRACT_STATUS_Signed']


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,35.0,0,0,1,0,0,0,0,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,69.0,0,0,1,0,0,0,0,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,30.0,0,0,1,0,0,0,0,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,10.0,0,0,1,0,0,0,0,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,101.0,0,0,1,0,0,0,0,0,0


In [36]:
data_cc_bal = data_cc_bal.drop(columns='SK_ID_PREV')

# Traitement des données groupées de la table data_cc_bal en utilisant les spécifications définies précédemment.
data_cc_bal_clean = data_cc_bal.groupby(ID).agg(['min', 'max', 'mean', 'sum', 'var'])

data_cc_bal_clean.columns = flatten_and_rename_columns(data_cc_bal_clean, prefix='CC_')
data_cc_bal_clean.head()

,CC_MONTHS_BALANCE_MIN,CC_MONTHS_BALANCE_MAX,CC_MONTHS_BALANCE_MEAN,CC_MONTHS_BALANCE_SUM,CC_MONTHS_BALANCE_VAR,CC_AMT_BALANCE_MIN,CC_AMT_BALANCE_MAX,CC_AMT_BALANCE_MEAN,CC_AMT_BALANCE_SUM,CC_AMT_BALANCE_VAR,...,CC_NAME_CONTRACT_STATUS_Sent proposal_MIN,CC_NAME_CONTRACT_STATUS_Sent proposal_MAX,CC_NAME_CONTRACT_STATUS_Sent proposal_MEAN,CC_NAME_CONTRACT_STATUS_Sent proposal_SUM,CC_NAME_CONTRACT_STATUS_Sent proposal_VAR,CC_NAME_CONTRACT_STATUS_Signed_MIN,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,-6,-1,-3.5,-21,3.5,0.0,0.00,0.000000,0.000,0.000000e+00,...,0,0,0.0,0,0.0,0,0,0.0,0,0.0
100011,-75,-2,-38.5,-2849,462.5,0.0,189000.00,54482.111149,4031676.225,4.641321e+09,...,0,0,0.0,0,0.0,0,0,0.0,0,0.0
100013,-96,-1,-48.5,-4656,776.0,0.0,161420.22,18159.919219,1743352.245,1.869473e+09,...,0,0,0.0,0,0.0,0,0,0.0,0,0.0
100021,-18,-2,-10.0,-170,25.5,0.0,0.00,0.000000,0.000,0.000000e+00,...,0,0,0.0,0,0.0,0,0,0.0,0,0.0
100023,-11,-4,-7.5,-60,6.0,0.0,0.00,0.000000,0.000,0.000000e+00,...,0,0,0.0,0,0.0,0,0,0.0,0,0.0


In [37]:
# Count credit card lines
data_cc_bal_clean['CC_COUNT'] = data_cc_bal.groupby(ID).size()

In [38]:
# Merge to data_train_encoded_copy
data_train_clean = data_train_clean.join(data_cc_bal_clean, how='left', on=ID)
data_train_clean.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_NAME_CONTRACT_STATUS_Sent proposal_MAX,CC_NAME_CONTRACT_STATUS_Sent proposal_MEAN,CC_NAME_CONTRACT_STATUS_Sent proposal_SUM,CC_NAME_CONTRACT_STATUS_Sent proposal_VAR,CC_NAME_CONTRACT_STATUS_Signed_MIN,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_COUNT
0,100002.0,1.0,0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003.0,0.0,0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004.0,0.0,1,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006.0,0.0,0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,100007.0,0.0,0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Table `installments_payments`

**1. Ajout de 4 caractéristiques :** `PAYMENT_PERC`, `PAYMENT_DIFF`, `DAYS_PAST_DUE` et `DAYS_BEFORE_DUE` :

* **`PAYMENT_PERC` :** le pourcentage du montant du paiement par rapport au montant dû.
* **`PAYMENT_DIFF` :** la différence entre le montant dû et le montant payé.
* **`DPD` :** le nombre de jours de retard du paiement par rapport à la date d'échéance.
* **`DBD` :** le nombre de jours avant la date d'échéance où le paiement a été effectué.

**2. Encodage One-hot des variables catégorielles :**

**3. Ajouter une caractéristique :** `INST_COUNT` (nombre de comptes de paiements échelonnés) : Cette caractéristique est calculée en comptant le nombre de comptes de paiements échelonnés pour chaque client dans la table installments_payments.

**4. Jointure de `installments_payments` avec `data_train_clean` par ID :** Après la préparation et l'enrichissement de la table installments_payments, celle-ci est fusionnée avec la table principale application_train en utilisant l'identifiant unique du client (`SK_ID_CURR`). Cette fusion permet d'intégrer les informations détaillées sur les paiements échelonnés des clients, améliorant ainsi la qualité du modèle prédictif pour évaluer le risque de défaut de crédit.

In [39]:
data_ins_payments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [40]:
# Percentage and difference paid in each installment (amount paid and installment value)
data_ins_payments['PAYMENT_PERC'] = data_ins_payments['AMT_PAYMENT'] / data_ins_payments['AMT_INSTALMENT']
data_ins_payments['PAYMENT_DIFF'] = data_ins_payments['AMT_INSTALMENT'] - data_ins_payments['AMT_PAYMENT']

# Days past due and days before due (no negative values). DPD = Days past due, DBD = days before due
data_ins_payments['DPD'] = data_ins_payments['DAYS_ENTRY_PAYMENT'] - data_ins_payments['DAYS_INSTALMENT']
data_ins_payments['DPD'] = data_ins_payments['DPD'].apply(lambda x: x if x > 0 else 0)
data_ins_payments['DBD'] = data_ins_payments['DAYS_INSTALMENT'] - data_ins_payments['DAYS_ENTRY_PAYMENT']
data_ins_payments['DBD'] = data_ins_payments['DBD'].apply(lambda x: x if x > 0 else 0)

data_ins_payments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,PAYMENT_PERC,PAYMENT_DIFF,DPD,DBD
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360,1.000000,0.000,0.0,7.0
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525,1.000000,0.000,0.0,0.0
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000,1.000000,0.000,0.0,0.0
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130,1.000000,0.000,0.0,8.0
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585,0.997942,4.455,17.0,0.0


In [41]:
# Features: Perform aggregations
agg_ins = {
    'NUM_INSTALMENT_VERSION': ['nunique'],
    'DPD': ['max', 'mean', 'sum'],
    'DBD': ['max', 'mean', 'sum'],
    'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
    'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
    'AMT_INSTALMENT': ['max', 'mean', 'sum'],
    'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
    'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
}

In [42]:
# Traitement des données groupées de la table bureau en utilisant les spécifications définies précédemment.
data_ins_payments_clean = data_ins_payments.groupby(ID).agg(agg_ins)

data_ins_payments_clean.columns = flatten_and_rename_columns(data_ins_payments_clean, prefix='INSTAL_')
data_ins_payments_clean.head()

,INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE,INSTAL_DPD_MAX,INSTAL_DPD_MEAN,INSTAL_DPD_SUM,INSTAL_DBD_MAX,INSTAL_DBD_MEAN,INSTAL_DBD_SUM,INSTAL_PAYMENT_PERC_MAX,INSTAL_PAYMENT_PERC_MEAN,INSTAL_PAYMENT_PERC_SUM,...,INSTAL_AMT_INSTALMENT_MAX,INSTAL_AMT_INSTALMENT_MEAN,INSTAL_AMT_INSTALMENT_SUM,INSTAL_AMT_PAYMENT_MIN,INSTAL_AMT_PAYMENT_MAX,INSTAL_AMT_PAYMENT_MEAN,INSTAL_AMT_PAYMENT_SUM,INSTAL_DAYS_ENTRY_PAYMENT_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MEAN,INSTAL_DAYS_ENTRY_PAYMENT_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,2,11.0,1.571429,11.0,36.0,8.857143,62.0,1.0,1.0,7.0,...,17397.900,5885.132143,41195.925,3951.000,17397.900,5885.132143,41195.925,-1628.0,-2195.000000,-15365.0
100002,2,0.0,0.000000,0.0,31.0,20.421053,388.0,1.0,1.0,19.0,...,53093.745,11559.247105,219625.695,9251.775,53093.745,11559.247105,219625.695,-49.0,-315.421053,-5993.0
100003,2,0.0,0.000000,0.0,14.0,7.160000,179.0,1.0,1.0,25.0,...,560835.360,64754.586000,1618864.650,6662.970,560835.360,64754.586000,1618864.650,-544.0,-1385.320000,-34633.0
100004,2,0.0,0.000000,0.0,11.0,7.666667,23.0,1.0,1.0,3.0,...,10573.965,7096.155000,21288.465,5357.250,10573.965,7096.155000,21288.465,-727.0,-761.666667,-2285.0
100005,2,1.0,0.111111,1.0,37.0,23.666667,213.0,1.0,1.0,9.0,...,17656.245,6240.205000,56161.845,4813.200,17656.245,6240.205000,56161.845,-470.0,-609.555556,-5486.0


In [43]:
# Count installments accounts
data_ins_payments_clean['INSTAL_COUNT'] = data_ins_payments.groupby(ID).size()

In [44]:
# Merge to application dataframe
data_train_clean = data_train_clean.join(data_ins_payments_clean, how='left', on=ID)
data_train_clean.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INSTAL_AMT_INSTALMENT_MEAN,INSTAL_AMT_INSTALMENT_SUM,INSTAL_AMT_PAYMENT_MIN,INSTAL_AMT_PAYMENT_MAX,INSTAL_AMT_PAYMENT_MEAN,INSTAL_AMT_PAYMENT_SUM,INSTAL_DAYS_ENTRY_PAYMENT_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MEAN,INSTAL_DAYS_ENTRY_PAYMENT_SUM,INSTAL_COUNT
0,100002.0,1.0,0,0,0,0,0,202500.0,406597.5,24700.5,...,11559.247105,219625.695,9251.775,53093.745,11559.247105,219625.695,-49.0,-315.421053,-5993.0,19.0
1,100003.0,0.0,0,1,0,1,0,270000.0,1293502.5,35698.5,...,64754.586000,1618864.650,6662.970,560835.360,64754.586000,1618864.650,-544.0,-1385.320000,-34633.0,25.0
2,100004.0,0.0,1,0,1,0,0,67500.0,135000.0,6750.0,...,7096.155000,21288.465,5357.250,10573.965,7096.155000,21288.465,-727.0,-761.666667,-2285.0,3.0
3,100006.0,0.0,0,1,0,0,0,135000.0,312682.5,29686.5,...,62947.088438,1007153.415,2482.920,691786.890,62947.088438,1007153.415,-12.0,-271.625000,-4346.0,16.0
4,100007.0,0.0,0,0,0,0,0,121500.0,513000.0,21865.5,...,12666.444545,835985.340,0.180,22678.785,12214.060227,806127.975,-14.0,-1032.242424,-68128.0,66.0


## Table `POS_cash_balance`

1. **Encodage One-hot des variables catégorielles**

2. **Ajout d'une caractéristique :** `POS_COUNT`(nombre de comptes de paiements par point de vente) : Cette nouvelle caractéristique représente le nombre total de comptes de paiements par point de vente associés à chaque client.

3. **Jointure de `pos_cash_balance` avec `data_train_clean` par `ID`**

In [45]:
data_pos_cash_bal.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [46]:
data_pos_cash_bal, cat_pos_bal_cols = one_hot_encoder(data_pos_cash_bal, nan_as_category=True)
data_pos_cash_bal.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Amortized debt,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_XNA,NAME_CONTRACT_STATUS_nan
0,1803195,182943,-31,48.0,45.0,0,0,1,0,0,0,0,0,0,0,0,0
1,1715348,367990,-33,36.0,35.0,0,0,1,0,0,0,0,0,0,0,0,0
2,1784872,397406,-32,12.0,9.0,0,0,1,0,0,0,0,0,0,0,0,0
3,1903291,269225,-35,48.0,42.0,0,0,1,0,0,0,0,0,0,0,0,0
4,2341044,334279,-35,36.0,35.0,0,0,1,0,0,0,0,0,0,0,0,0


In [47]:
# Features: Perform aggregations
agg_pos = {
    'MONTHS_BALANCE': ['max', 'mean', 'size'],
    'SK_DPD': ['max', 'mean'],
    'SK_DPD_DEF': ['max', 'mean']
}

for cat in cat_pos_bal_cols:
    agg_pos[cat] = ['mean']
print(agg_pos)

{'MONTHS_BALANCE': ['max', 'mean', 'size'], 'SK_DPD': ['max', 'mean'], 'SK_DPD_DEF': ['max', 'mean'], 'NAME_CONTRACT_STATUS_Active': ['mean'], 'NAME_CONTRACT_STATUS_Amortized debt': ['mean'], 'NAME_CONTRACT_STATUS_Approved': ['mean'], 'NAME_CONTRACT_STATUS_Canceled': ['mean'], 'NAME_CONTRACT_STATUS_Completed': ['mean'], 'NAME_CONTRACT_STATUS_Demand': ['mean'], 'NAME_CONTRACT_STATUS_Returned to the store': ['mean'], 'NAME_CONTRACT_STATUS_Signed': ['mean'], 'NAME_CONTRACT_STATUS_XNA': ['mean'], 'NAME_CONTRACT_STATUS_nan': ['mean']}


In [48]:
# Traitement des données groupées de la table bureau en utilisant les spécifications définies précédemment.
data_pos_cash_bal_clean = data_pos_cash_bal.groupby(ID).agg(agg_pos)

data_pos_cash_bal_clean.columns = flatten_and_rename_columns(data_pos_cash_bal_clean, prefix='INSTAL_')
data_pos_cash_bal_clean.head()

,INSTAL_MONTHS_BALANCE_MAX,INSTAL_MONTHS_BALANCE_MEAN,INSTAL_MONTHS_BALANCE_SIZE,INSTAL_SK_DPD_MAX,INSTAL_SK_DPD_MEAN,INSTAL_SK_DPD_DEF_MAX,INSTAL_SK_DPD_DEF_MEAN,INSTAL_NAME_CONTRACT_STATUS_Active_MEAN,INSTAL_NAME_CONTRACT_STATUS_Amortized debt_MEAN,INSTAL_NAME_CONTRACT_STATUS_Approved_MEAN,INSTAL_NAME_CONTRACT_STATUS_Canceled_MEAN,INSTAL_NAME_CONTRACT_STATUS_Completed_MEAN,INSTAL_NAME_CONTRACT_STATUS_Demand_MEAN,INSTAL_NAME_CONTRACT_STATUS_Returned to the store_MEAN,INSTAL_NAME_CONTRACT_STATUS_Signed_MEAN,INSTAL_NAME_CONTRACT_STATUS_XNA_MEAN,INSTAL_NAME_CONTRACT_STATUS_nan_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,
100001,-53,-72.555556,9,7,0.777778,7,0.777778,0.777778,0.0,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0
100002,-1,-10.000000,19,0,0.000000,0,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
100003,-18,-43.785714,28,0,0.000000,0,0.000000,0.928571,0.0,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0
100004,-24,-25.500000,4,0,0.000000,0,0.000000,0.750000,0.0,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0
100005,-15,-20.000000,11,0,0.000000,0,0.000000,0.818182,0.0,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0


In [49]:
# Count installments accounts
data_pos_cash_bal_clean['POS_COUNT'] = data_pos_cash_bal.groupby(ID).size()

In [50]:
data_train_clean = data_train_clean.join(data_pos_cash_bal_clean, how='left', on=ID)
data_train_clean.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INSTAL_NAME_CONTRACT_STATUS_Amortized debt_MEAN,INSTAL_NAME_CONTRACT_STATUS_Approved_MEAN,INSTAL_NAME_CONTRACT_STATUS_Canceled_MEAN,INSTAL_NAME_CONTRACT_STATUS_Completed_MEAN,INSTAL_NAME_CONTRACT_STATUS_Demand_MEAN,INSTAL_NAME_CONTRACT_STATUS_Returned to the store_MEAN,INSTAL_NAME_CONTRACT_STATUS_Signed_MEAN,INSTAL_NAME_CONTRACT_STATUS_XNA_MEAN,INSTAL_NAME_CONTRACT_STATUS_nan_MEAN,POS_COUNT
0,100002.0,1.0,0,0,0,0,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0
1,100003.0,0.0,0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0
2,100004.0,0.0,1,0,1,0,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0
3,100006.0,0.0,0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0
4,100007.0,0.0,0,0,0,0,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0


## Table `previous_application`

1. **Remplacement des valeurs anormales `DAYS_`... par NaN :** Les anomalies dans les variables temporelles, telles que les jours écoulés depuis un événement précédent, sont remplacées par des valeurs NaN pour assurer une cohérence et une fiabilité des données, évitant ainsi des erreurs de modèle dues à des valeurs aberrantes.

2. **Création d'une caractéristique :** `APP_CREDIT_PERC` (pourcentage du crédit de l'application) : Cette nouvelle caractéristique représente le nombre total de comptes de paiements par point de vente associés à chaque client.

3. **Encodage One-hot des variables catégorielles**

4. **Jointure de `previous_application` avec `data_train_clean` par `ID`**

In [51]:
data_prev_app.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
days_cols = ['DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION']
data_prev_app[days_cols].describe()

,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION
count,997149.000000,997149.000000,997149.000000,997149.000000,997149.000000
mean,342209.855039,13826.269337,33767.774054,76582.403064,81992.343838
std,88916.115834,72444.869708,106857.034789,149647.415123,153303.516729
min,-2922.000000,-2892.000000,-2801.000000,-2889.000000,-2874.000000
25%,365243.000000,-1628.000000,-1242.000000,-1314.000000,-1270.000000
50%,365243.000000,-831.000000,-361.000000,-537.000000,-499.000000
75%,365243.000000,-411.000000,129.000000,-74.000000,-44.000000
max,365243.000000,365243.000000,365243.000000,365243.000000,365243.000000


In [53]:
# Identify the maximum value in 'DAYS_FIRST_DRAWING', which is often used as a placeholder for missing or not applicable data
anom_days_first_drawing = data_prev_app['DAYS_FIRST_DRAWING'].max()

# Replace the identified anomalous maximum value with NaN to handle improper or extreme values that can skew analysis
data_prev_app[days_cols] = data_prev_app[days_cols].replace({anom_days_first_drawing: np.nan})

print(f"DAYS_colonnes: remplacé {anom_days_first_drawing} par NaN dans les colonnes: {', '.join(days_cols)}")

DAYS_colonnes: remplacé 365243.0 par NaN dans les colonnes: DAYS_FIRST_DRAWING, DAYS_FIRST_DUE, DAYS_LAST_DUE_1ST_VERSION, DAYS_LAST_DUE, DAYS_TERMINATION


In [54]:
data_prev_app[days_cols].describe()

,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION
count,62705.000000,956504.000000,903285.000000,785928.000000,771236.000000
mean,-1035.246791,-1106.583027,-677.158175,-996.179128,-978.375222
std,922.710316,790.703113,923.601152,752.703178,749.134297
min,-2922.000000,-2892.000000,-2801.000000,-2889.000000,-2874.000000
25%,-1721.000000,-1676.000000,-1359.000000,-1566.000000,-1539.000000
50%,-621.000000,-874.000000,-480.000000,-801.000000,-780.000000
75%,-303.000000,-459.000000,-2.000000,-353.000000,-337.000000
max,-2.000000,-2.000000,2389.000000,-2.000000,-2.000000


Le traitement des valeurs anormales dans les caractéristiques liées au temps, comme les jours jusqu'à la prochaine échéance de paiement, est un aspect crucial du prétraitement des données

* **Réduction du Nombre de Données (count) :** La diminution considérable du nombre de données observables pour `DAYS_FIRST_DRAWING` indique que beaucoup de valeurs de cette colonne étaient des anomalies et ont été remplacées par NaN. Les autres colonnes montrent également une légère réduction ce qui signifie que l'anomalie était particulièrement concentrée dans `DAYS_FIRST_DRAWING`.

* **Changement de la Moyenne (mean) :** La moyenne est devenue beaucoup plus raisonnable et représente des valeurs typiques de jours dans le contexte de prêts. Les valeurs extrêmes n'influencent plus cette statistique, donnant une meilleure compréhension de la distribution temporelle des événements liés au prêt.

* **Réduction de l'Écart-Type (std) :** L'écart-type a considérablement diminué dans toutes les colonnes, ce qui signifie que les données sont maintenant moins dispersées et plus cohérentes. Cela suggère que les valeurs extrêmes étaient une source majeure de variabilité.

* **Ajustement des Valeurs Minimales et Maximales (min et max) :** La valeur maximale de plusieurs colonnes n'est plus l'anomalie, reflétant une gamme de valeurs plus typique et réelle pour des caractéristiques associées au temps.

* **Changement des Quartiles :** Les quartiles ont tous changé pour montrer un intervalle de valeurs plus serré, aligné avec ce qu'on attendrait logiquement de délais dans un contexte de prêts.

In [55]:
data_prev_app['APP_CREDIT_PERC'] = data_prev_app['AMT_APPLICATION'] / data_prev_app['AMT_CREDIT']

In [56]:
data_prev_app, cat_prev_cols = one_hot_encoder(data_prev_app, nan_as_category=True)
data_prev_app.head()

,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,...,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest,PRODUCT_COMBINATION_nan
0,2030495,271877,1730.430,17145.0,17145.0,0.0,17145.0,15,1,0.0,...,0,0,0,0,0,1,0,0,0,0
1,2802425,108129,25188.615,607500.0,679671.0,NaN,607500.0,11,1,NaN,...,0,0,0,0,0,0,0,0,0,0
2,2523466,122040,15060.735,112500.0,136444.5,NaN,112500.0,11,1,NaN,...,0,0,0,0,0,0,0,0,0,0
3,2819243,176158,47041.335,450000.0,470790.0,NaN,450000.0,7,1,NaN,...,1,0,0,0,0,0,0,0,0,0
4,1784265,202054,31924.395,337500.0,404055.0,NaN,337500.0,9,1,NaN,...,0,0,0,0,0,0,0,0,0,0


In [57]:
# Previous applications numeric features
agg_prev = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }

# Previous applications categorical features
for cat in cat_prev_cols:
    agg_prev[cat] = ['mean']

In [58]:
data_prev_app_clean = data_prev_app.groupby(ID).agg(agg_prev)
data_prev_app_clean.columns = flatten_and_rename_columns(data_prev_app_clean, prefix='PREV_')
data_prev_app_clean.head()

,PREV_AMT_ANNUITY_MIN,PREV_AMT_ANNUITY_MAX,PREV_AMT_ANNUITY_MEAN,PREV_AMT_APPLICATION_MIN,PREV_AMT_APPLICATION_MAX,PREV_AMT_APPLICATION_MEAN,PREV_AMT_CREDIT_MIN,PREV_AMT_CREDIT_MAX,PREV_AMT_CREDIT_MEAN,PREV_APP_CREDIT_PERC_MIN,...,PREV_PRODUCT_COMBINATION_Cash X-Sell: middle_MEAN,PREV_PRODUCT_COMBINATION_POS household with interest_MEAN,PREV_PRODUCT_COMBINATION_POS household without interest_MEAN,PREV_PRODUCT_COMBINATION_POS industry with interest_MEAN,PREV_PRODUCT_COMBINATION_POS industry without interest_MEAN,PREV_PRODUCT_COMBINATION_POS mobile with interest_MEAN,PREV_PRODUCT_COMBINATION_POS mobile without interest_MEAN,PREV_PRODUCT_COMBINATION_POS other with interest_MEAN,PREV_PRODUCT_COMBINATION_POS others without interest_MEAN,PREV_PRODUCT_COMBINATION_nan_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3951.000,3951.000,3951.000,24835.5,24835.5,24835.50,23787.0,23787.0,23787.00,1.044079,...,0.0,0.000000,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0
100002,9251.775,9251.775,9251.775,179055.0,179055.0,179055.00,179055.0,179055.0,179055.00,1.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0
100003,6737.310,98356.995,56553.990,68809.5,900000.0,435436.50,68053.5,1035882.0,484191.00,0.868825,...,0.0,0.333333,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0
100004,5357.250,5357.250,5357.250,24282.0,24282.0,24282.00,20106.0,20106.0,20106.00,1.207699,...,0.0,0.000000,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0
100005,4813.200,4813.200,4813.200,0.0,44617.5,22308.75,0.0,40153.5,20076.75,1.111173,...,0.0,0.000000,0.0,0.000000,0.0,0.5,0.0,0.0,0.0,0.0


In [59]:
data_train_clean = data_train_clean.join(data_prev_app_clean, how='left', on=ID)
data_train_clean.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,PREV_PRODUCT_COMBINATION_Cash X-Sell: middle_MEAN,PREV_PRODUCT_COMBINATION_POS household with interest_MEAN,PREV_PRODUCT_COMBINATION_POS household without interest_MEAN,PREV_PRODUCT_COMBINATION_POS industry with interest_MEAN,PREV_PRODUCT_COMBINATION_POS industry without interest_MEAN,PREV_PRODUCT_COMBINATION_POS mobile with interest_MEAN,PREV_PRODUCT_COMBINATION_POS mobile without interest_MEAN,PREV_PRODUCT_COMBINATION_POS other with interest_MEAN,PREV_PRODUCT_COMBINATION_POS others without interest_MEAN,PREV_PRODUCT_COMBINATION_nan_MEAN
0,100002.0,1.0,0,0,0,0,0,202500.0,406597.5,24700.5,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.0,0.0,0.0
1,100003.0,0.0,0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.0,0.333333,0.0,0.333333,0.0,0.000000,0.0,0.0,0.0,0.0
2,100004.0,0.0,1,0,1,0,0,67500.0,135000.0,6750.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.0,0.0,0.0,0.0
3,100006.0,0.0,0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.111111,0.0,0.111111,0.0,0.000000,0.0,0.0,0.0,0.0
4,100007.0,0.0,0,0,0,0,0,121500.0,513000.0,21865.5,...,0.5,0.166667,0.0,0.000000,0.0,0.166667,0.0,0.0,0.0,0.0


# Préparation des Données Finales pour la Modélisation

Après avoir réalisé le feature engineering, la prochaine étape consiste à préparer le jeu de données final qui sera utilisé pour entraîner notre modèle. Cette étape implique la consolidation de toutes les transformations, normalisations, et encodages appliqués aux données initiales pour former un ensemble cohérent et optimisé pour le machine learning.

## Correlation des varriables

In [60]:
# Calcul de la matrice de corrélation sur les données, sans inclure les identifiants et la cible
corr = data_train_clean.drop(columns=[ID, TARGET]).corr().abs()

In [61]:
# Sélection de la partie supérieure de la matrice de corrélation
upper_corr = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
display(upper_corr)

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,PREV_PRODUCT_COMBINATION_Cash X-Sell: middle_MEAN,PREV_PRODUCT_COMBINATION_POS household with interest_MEAN,PREV_PRODUCT_COMBINATION_POS household without interest_MEAN,PREV_PRODUCT_COMBINATION_POS industry with interest_MEAN,PREV_PRODUCT_COMBINATION_POS industry without interest_MEAN,PREV_PRODUCT_COMBINATION_POS mobile with interest_MEAN,PREV_PRODUCT_COMBINATION_POS mobile without interest_MEAN,PREV_PRODUCT_COMBINATION_POS other with interest_MEAN,PREV_PRODUCT_COMBINATION_POS others without interest_MEAN,PREV_PRODUCT_COMBINATION_nan_MEAN
NAME_CONTRACT_TYPE,NaN,0.006553,0.004186,0.063077,0.028722,0.004817,0.200180,0.228259,0.166608,0.023615,...,0.046572,0.019529,0.025580,0.030602,0.018561,0.032114,0.048669,0.027558,0.017126,0.001297
CODE_GENDER,NaN,NaN,0.343453,0.041581,0.048262,0.078535,0.016907,0.076729,0.018231,0.013552,...,0.061502,0.056363,0.024400,0.086894,0.036346,0.048137,0.028259,0.027166,0.015572,0.003205
FLAG_OWN_CAR,NaN,NaN,NaN,0.002101,0.103053,0.089024,0.116985,0.143507,0.121231,0.039093,...,0.050634,0.018492,0.028470,0.030213,0.000069,0.034950,0.030911,0.032496,0.011882,0.000071
FLAG_OWN_REALTY,NaN,NaN,NaN,NaN,0.001882,0.004256,0.034738,0.001626,0.042337,0.018158,...,0.019749,0.024256,0.011362,0.024484,0.011994,0.053379,0.000818,0.008636,0.003102,0.001723
CNT_CHILDREN,NaN,NaN,NaN,NaN,NaN,0.014070,0.005996,0.025998,0.002254,0.024210,...,0.070155,0.031961,0.014897,0.015075,0.005337,0.074288,0.027689,0.003282,0.006778,0.001632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PREV_PRODUCT_COMBINATION_POS mobile with interest_MEAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.035722,0.072403,0.024576,0.002902
PREV_PRODUCT_COMBINATION_POS mobile without interest_MEAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.024889,0.008430,0.001149
PREV_PRODUCT_COMBINATION_POS other with interest_MEAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005040,0.001902
PREV_PRODUCT_COMBINATION_POS others without interest_MEAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000364


In [62]:
# Identification des variables avec une corrélation supérieure à 0.98
cols_to_drop = [column for column in upper_corr.columns if any(upper_corr[column] > 0.98)]
print(f"Colonnes à supprimer dû à la forte correlation: {cols_to_drop}")
print(f"Nombre de colonnes à supprimer: {len(cols_to_drop)}")

Colonnes à supprimer dû à la forte correlation: ['AMT_GOODS_PRICE', 'YEARS_BUILD_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 'OBS_60_CNT_SOCIAL_CIRCLE', 'NAME_INCOME_TYPE_Pensioner', 'ORGANIZATION_TYPE_XNA', 'BURO_MONTHS_BALANCE_MIN_MIN', 'BURO_CREDIT_ACTIVE_Closed_MEAN', 'CC_MONTHS_BALANCE_MEAN', 'CC_MONTHS_BALANCE_SUM', 'CC_MONTHS_BALANCE_VAR', 'CC_AMT_PAYMENT_TOTAL_CURRENT_MAX', 'CC_AMT_PAYMENT_TOTAL_CURRENT_SUM', 'CC_AMT_PAYMENT_TOTAL_CURRENT_VAR', 'CC_AMT_RECEIVABLE_PRINCIPAL_MIN', 'CC_AMT_RECEIVABLE_PRINCIPAL_MAX', 'CC_AMT_RECEIVABLE_PRINCIPAL_MEAN', 'CC_AMT_RECEIVABLE_PRINCIPAL_SUM', 'CC_AMT_RECEIVABLE_PRINCIPAL_VAR', 'CC_AMT_RECIVABLE_MIN', 'CC_AMT_RECIVABLE_MAX', 'CC_AMT_RECIVABLE_MEAN', '

In [63]:
# Suppression des variables fortement corrélées
data_train_clean = data_train_clean.drop(columns=cols_to_drop)
data_train_clean.shape

(356249, 599)

## Données finales : processed_data

1. **Vérification des valeurs manquantes :** Je vais calculer le nombre total et le pourcentage de valeurs manquantes dans notre jeu de données nettoyé.

2. **Vérification des valeurs infinies :** je vais identifier et compter les valeurs infinies. Même si leur présence est relativement rare (0.00% de notre jeu de données), il est crucial de les traiter car elles peuvent causer des erreurs dans de nombreux algorithmes de machine learning.

3. **Remplacement des valeurs infinies :** Je vais remplacer les valeurs infinies par NaN (np.nan). Cela standardise notre traitement des valeurs anormales et permet d'appliquer ensuite des stratégies d'imputation cohérentes sur l'ensemble de notre jeu de données.

4. **Exporter `processed_data`pour la modélisation**

In [64]:
data_train_clean.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,PREV_PRODUCT_COMBINATION_Cash X-Sell: low_MEAN,PREV_PRODUCT_COMBINATION_Cash X-Sell: middle_MEAN,PREV_PRODUCT_COMBINATION_POS household with interest_MEAN,PREV_PRODUCT_COMBINATION_POS household without interest_MEAN,PREV_PRODUCT_COMBINATION_POS industry with interest_MEAN,PREV_PRODUCT_COMBINATION_POS industry without interest_MEAN,PREV_PRODUCT_COMBINATION_POS mobile with interest_MEAN,PREV_PRODUCT_COMBINATION_POS mobile without interest_MEAN,PREV_PRODUCT_COMBINATION_POS other with interest_MEAN,PREV_PRODUCT_COMBINATION_POS others without interest_MEAN
0,100002.0,1.0,0,0,0,0,0,202500.0,406597.5,24700.5,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.0,0.0
1,100003.0,0.0,0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.333333,0.0,0.333333,0.0,0.333333,0.0,0.000000,0.0,0.0,0.0
2,100004.0,0.0,1,0,1,0,0,67500.0,135000.0,6750.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.0,0.0,0.0
3,100006.0,0.0,0,1,0,0,0,135000.0,312682.5,29686.5,...,0.222222,0.0,0.111111,0.0,0.111111,0.0,0.000000,0.0,0.0,0.0
4,100007.0,0.0,0,0,0,0,0,121500.0,513000.0,21865.5,...,0.000000,0.5,0.166667,0.0,0.000000,0.0,0.166667,0.0,0.0,0.0


In [65]:
# Vérification des valeurs manquantes
nan_val = data_train_clean.isna().sum().sum()
print(f"Valeurs manquantes = {nan_val} ({nan_val / data_train_clean.size:.2%})")

# Vérification des valeurs infinies
infinite_val = np.isinf(data_train_clean).values.sum()
print(f"Valeurs infinies = {infinite_val} ({infinite_val / data_train_clean.size:.2%})")

Valeurs manquantes = 40861843 (19.15%)
Valeurs infinies = 20 (0.00%)


In [66]:
# Remplacement des valeurs infinies
data_train_clean = data_train_clean.replace([np.inf, -np.inf], np.nan)

# Exportation des Jeux de Données pour l'Entraînement et le Test

In [67]:
# Division du jeu de données
# Séparation du jeu de données en deux parties : une pour l'entraînement et une pour les tests.
processed_data_test = data_train_clean[data_train_clean[ID].isin(list_ids)]
processed_data_train = data_train_clean[~data_train_clean[ID].isin(list_ids)]

In [68]:
# Exportation des jeux de données sans la colonne ID
# Pour le jeu de données de test, on exporte sans les colonnes ID et TARGET pour prévenir des fuites de données lors de la phase de test.
processed_data_test.drop(columns=[ID, TARGET]).to_csv(path_data_processed + 'processed_data_test.csv')
processed_data_train.drop(columns=ID).to_csv(path_data_processed + 'processed_data_train.csv')